# Welcom to SSCMA for Google Colab Training Example 🔥

<a href="https://colab.research.google.com/github/Seeed-Studio/SSCMA/blob/main/notebooks/Google-Colab-SWIFT-YOLO-A1101-Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> **[🚀🚀🚀 One-Click to Deploy to Google Colab 🚀🚀🚀](https://colab.research.google.com/github/Seeed-Studio/SSCMA/blob/main/notebooks/Google-Colab-SWIFT-YOLO-A1101-Example.ipynb)**

In this tutorial, we will demonstrate how to use [SSCMA](https://github.com/Seeed-Studio/SSCMA) and Colab to train a SWFIT-YOLO model for meter reading ⏱️.

Table of contents:

- [Setup SSCMA](#setup-SSCMA)
- [Download Custom Meter Datasets](#download-custom-meter-datasets)
- [Train SWFIT-YOLO Model](#train-yolov5-tiny-model)
- [Optimize SWFIT-YOLO Model and Export](#optimize-yolov5-tiny-model-and-export)
- [Deploy to SenseCAP A1101](#deploy-to-sensecap-a1101)
- [Thanks for Trying out SSCMA 🎉](#thanks-for-trying-out-SSCMA-🎉)

For more details of SWIFT-YOLO model or application scenarios, please refer to [SSCMA Documentation](https://seeed-studio.github.io/SSCMA/tutorials/training/yolov5).

**Tips: Since model training consumes a lot of computational resources, we recommend that you run this notebook on [Colab's GPU instances](https://research.google.com/colaboratory/faq.html#gpu-availability).**

## Setup sscma

**Step 0:** Clone the SSCMA source code from [SSCMA GitHub repository](https://github.com/Seeed-Studio/SSCMA), and enter the SSCMA project directory.

In [ ]:
# Ethos-U-Vela need to be installed this way, or SSCMA does not work anymore...
!git clone https://review.mlplatform.org/ml/ethos-u/ethos-u-vela.git%cd ethos-u-vela!pip install .%cd..!git clone https://github.com/Seeed-Studio/SSCMA.git  # currently we're using experimental 2.0 version branch
%cd /content/SSCMA

**Step 1:** Install python third-party library

In [ ]:
!pip install .

## Download Custom Meter Datasets

We'll download our dataset from Roboflow. Use the "**COCO**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).
<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="480" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/6152a275ad4b4ac20cd2e21a_roboflow-annotate.gif"/></a></p>Label images lightning fast (including with model-assisted labeling)

- open our digital meter dataset [seeed_meter_digit](https://universe.roboflow.com/seeeddatasets/seeed_meter_digit/)
- Download Dataset with COCO format
- Copy the download code and paste it into the below

***More Seeed Studio public datasets are available [here](https://universe.roboflow.com/seeeddatasets).***

In [ ]:
!mkdir -p datasets
!wget "https://universe.roboflow.com/ds/GDH5ex194I?key=dPovYB5xI6" -O datasets/digital_meter.zip
!unzip -q datasets/digital_meter.zip -d datasets/digital_meter && rm datasets/digital_meter.zip

## Train YOLOv5 Tiny Model

### Examine the YOLOv5 Tiny config file

The config we use for train is located at `configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py`.

For more config details, please refer to [SSCMA Documentation - Config](https://seeed-studio.github.io/SSCMA/tutorials/config).

In [ ]:
!cat configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py

### Train the model using default config

Then, train YOLOv5-Tiny model using default config, note here we need to override the config by 2 parameters:

- `data_root` - the datasets path, which located at path `datasets/digital_meter`

- `epochs`- the train epochs, we use `50` to reduce the training time

- `num_classes` - the classes number of datasets, we use `11` here ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'N']

For more training details, please refer to [SSCMA Documentation - Train YOLOv5-Tiny Model](https://seeed-studio.github.io/SSCMA/tutorials/training/yolov5).

In [ ]:
!python tools/train.py \
  configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
  --cfg-options \
    epochs=10 \
    num_classes=11 \
    data_root='datasets/digital_meter/'

The output directory in the training process would be look like this:

In [ ]:
!ls -alh work_dirs/swift_yolo_tiny_1xb16_300e_coco.py

We could inspect the latest model weight path by this command:

In [ ]:
!echo "Latest model weights path: $(cat work_dirs/swift_yolo_tiny_1xb16_300e_coco.py/last_checkpoint)"

### Validate the Model

We recommend you to validate the model with the latest weight after training using the `tool/test.py` script.

In [ ]:
!python tools/inference.py \
    configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
    "$(cat work_dirs/swift_yolo_tiny_1xb16_300e_coco.py/last_checkpoint)" \
    --dump work_dirs/swift_yolo_tiny_1xb16_300e_coco.py/last_checkpoint.pkl \
    --cfg-options \
        data_root='datasets/digital_meter/'\
        num_classes=11

## Optimize YOLOv5-Tiny Model and Export

### Export YOLOv5-Tiny PyTorch model to TFLite model

Here we're using `tools/torch2tflite.py` to convert and export the latest weights to a TFLite model at a INT8 precision for inference.

In [ ]:
!python tools/export.py \
  configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
  $(cat work_dirs/swift_yolo_tiny_1xb16_300e_coco.py/last_checkpoint) \
  --cfg-options \
    data_root='datasets/digital_meter/' \
    num_classes=11

## Deploy to SenseCAP A1101

This example is a tutorial for deploying the models from [SSCMA](https://github.com/Seeed-Studio/SSCMA/) to Grove - Vision AI module, based on the [Synopsys GUN Toolchain](https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain) and [Tensorflow Lite Micro](https://github.com/tensorflow/tflite-micro) implementations.

For more details, please refer to [SSCMA Documentation - Example - Grove - Deploy](https://sensecraftma.seeed.cc/SSCMA/examples/grove/deploy#compile-and-deploy).

### Download and Setup Build Tool Chain

In [ ]:
!wget https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain/releases/download/arc-2020.09-release/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz -P ~/

In [ ]:
!tar -zxf ~/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz --directory ~/

In [ ]:
SYSTEM_PATH = %env PATH
SYSTEM_PATH = SYSTEM_PATH.replace('"', '')
SYSTEM_HOME = %env HOME
SYSTEM_HOME = SYSTEM_HOME.replace('"', '')
SYSTEM_PATH = f'{SYSTEM_HOME}/arc_gnu_2020.09_prebuilt_elf32_le_linux_install/bin:{SYSTEM_PATH}'
%set_env PATH={SYSTEM_PATH}

### Clone and Setup SenseCAP A1101 SDK

In [ ]:
!git clone https://github.com/Seeed-Studio/sscma-example-vision-ai example/grove

In [ ]:
!python -m pip install numpy requests colorama serial pyserial

### Build Firmware

In [ ]:
!cd example/grove && \
  make HW=sensecap_vision_ai APP=digital_meter && make flash

### Convert Firmware Image to UF2

In [ ]:
!cd example/grove && \
  python tools/ufconv/uf2conv.py \
    -t 0 \
    -c tools/image_gen_cstm/output/output.img \
    -o firmware.uf2

In [ ]:
!echo "The UF2 firmware is located at: $(pwd)/example/grove/firmware.uf2"

In [ ]:
from IPython.display import FileLink

FileLink('example/grove/firmware.uf2')

### Convert Trained Model (TFLite) to UF2

In [ ]:
!cd example/grove && \
  python tools/ufconv/uf2conv.py \
    -t 18 \
    -c "$(cat ../../work_dirs/swift_yolo_tiny_1xb16_300e_coco.py/last_checkpoint | sed -e 's/.pth/_int8.tflite/g')" \
    -o model.uf2

In [ ]:
!echo "The UF2 model is located at: $(pwd)/example/grove/model.uf2"

In [ ]:
FileLink('example/grove/model.uf2')

Please download the UF2 Firmware and UF2 Model and flash them into a SenseCAP A1101 module. Finally, you can see the real-time meter reading results as shown in the figure below.

![Meter Reader](https://seeed-studio.github.io/SSCMA/static/grove/images/digital_meter.gif)

For more information on **how to use these UF2 images**, please refer to [SSCMA Documentations - Deploy - Grove - Deployment Routines](https://seeed-studio.github.io/sscma/examples/grove/deploy#deployment-routines).

## Thanks for Trying Out SSCMA 🎉

Congratulations, you have completed this tutorial. If you are interested in more application scenarios or our projects, please feel free to give [SSCMA](https://github.com/Seeed-Studio/SSCMA) a star ✨ on GitHub.

If you have any questions about this tutorial, please also feel free to [submit an issue](https://github.com/Seeed-Studio/SSCMA/issues).